In [1]:
# Import the quantum chemistry driver and related units
from qiskit_nature.units import DistanceUnit
from qiskit_nature.second_q.drivers import PySCFDriver

# Import the qubit mapper library for Ci+ and Ci
from qiskit_nature.second_q.mappers import JordanWignerMapper, BravyiKitaevMapper, BravyiKitaevSuperFastMapper

# Import a classical analyzer to compare results with VQE
from qiskit_algorithms import NumPyMinimumEigensolver

# Import VQE and related libraries
from qiskit_algorithms import VQE
from qiskit_algorithms.optimizers import SLSQP, NELDER_MEAD
from qiskit.primitives import Estimator
from qiskit_nature.second_q.circuit.library import HartreeFock, UCCSD

# Import ground state solver
from qiskit_nature.second_q.algorithms import GroundStateEigensolver

# For plotting
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
driver = PySCFDriver(
    atom="H 0.0 0.0 0.0; Li 0.0 0.0 1.596",
    basis="sto3g",
    charge=0,
    spin=0,
    unit=DistanceUnit.ANGSTROM,
)

es_problem = driver.run()

In [3]:
mapper = JordanWignerMapper()

In [4]:
ansatz = UCCSD(
    es_problem.num_spatial_orbitals,
    es_problem.num_particles,
    mapper,
    initial_state=HartreeFock(
        es_problem.num_spatial_orbitals,
        es_problem.num_particles,
        mapper,
    ),
)

vqe_solver = VQE(Estimator(), ansatz, NELDER_MEAD())
vqe_solver.initial_point = [0.0] * ansatz.num_parameters

In [5]:
ansatz.decompose().draw()

┌───┐┌────────────────────────────────────────────────┐»
 q_0: ┤ X ├┤0                                               ├»
      ├───┤│                                                │»
 q_1: ┤ X ├┤1                                               ├»
      └───┘│                                                │»
 q_2: ─────┤2                                               ├»
           │                                                │»
 q_3: ─────┤3                                               ├»
           │                                                │»
 q_4: ─────┤4                                               ├»
           │                                                │»
 q_5: ─────┤5                                               ├»
      ┌───┐│   exp(-it (IIIIIIIIIXZY + IIIIIIIIIYZX))(t[0]) │»
 q_6: ┤ X ├┤6                                               ├»
      ├───┤│                                                │»
 q_7: ┤ X ├┤7                                               ├»
      └───┘│                                                │»
 q_8: ─────┤8                                               ├»
           │                                                │»
 q_9: ─────┤9                                               ├»
           │                                                │»
q_10: ─────┤10                                              ├»
           │                                                │»
q_11: ─────┤11                                              ├»
           └────────────────────────────────────────────────┘»
«      ┌────────────────────────────────────────────────┐»
« q_0: ┤0                                               ├»
«      │                                                │»
« q_1: ┤1                                               ├»
«      │                                                │»
« q_2: ┤2                                               ├»
«      │                                                │»
« q_3: ┤3                                               ├»
«      │                                                │»
« q_4: ┤4                                               ├»
«      │                                                │»
« q_5: ┤5                                               ├»
«      │   exp(-it (IIIIIIIIXZZY + IIIIIIIIYZZX))(t[1]) │»
« q_6: ┤6                                               ├»
«      │                                                │»
« q_7: ┤7                                               ├»
«      │                                                │»
« q_8: ┤8                                               ├»
«      │                                                │»
« q_9: ┤9                                               ├»
«      │                                                │»
«q_10: ┤10                                              ├»
«      │                                                │»
«q_11: ┤11                                              ├»
«      └────────────────────────────────────────────────┘»
«      ┌────────────────────────────────────────────────┐»
« q_0: ┤0                                               ├»
«      │                                                │»
« q_1: ┤1                                               ├»
«      │                                                │»
« q_2: ┤2                                               ├»
«      │                                                │»
« q_3: ┤3                                               ├»
«      │                                                │»
« q_4: ┤4                                               ├»
«      │                                                │»
« q_5: ┤5                                               ├»
«      │   exp(-it (IIIIIIIXZZZY + IIIIIIIYZZZX))(t[2]) │»
« q_6: ┤6                                               ├»
«      │                                                │»
« q_7: ┤7                                               ├»
«      │    

In [6]:
calc = GroundStateEigensolver(mapper, vqe_solver)

In [ ]:
%%time
res = calc.solve(es_problem)
print(res)

### Comparing with a classical Eigensolver (provided by numpy)

In [4]:
numpy_solver = NumPyMinimumEigensolver()
calc2 = GroundStateEigensolver(mapper, numpy_solver)

In [5]:
%%time
res2 = calc2.solve(es_problem)
print(res2)

=== GROUND STATE ENERGY ===
 
* Electronic ground state energy (Hartree): -8.877080999128
  - computed part:      -8.877080999128
~ Nuclear repulsion energy (Hartree): 0.994694005489
> Total ground state energy (Hartree): -7.882386993639
 
=== MEASURED OBSERVABLES ===
 
  0:  # Particles: 4.000 S: 0.000 S^2: 0.000 M: 0.000
 
=== DIPOLE MOMENTS ===
 
~ Nuclear dipole moment (a.u.): [0.0  0.0  9.04800868]
 
  0: 
  * Electronic dipole moment (a.u.): [0.0  0.0  7.230352280595]
    - computed part:      [0.0  0.0  7.230352280595]
  > Dipole moment (a.u.): [0.0  0.0  1.817656399405]  Total: 1.817656399405
                 (debye): [0.0  0.0  4.620021302539]  Total: 4.620021302539
 
CPU times: user 4.75 s, sys: 114 ms, total: 4.86 s
Wall time: 4.25 s


### Measuring performance of UCCSD with various interactomic distance